In [36]:
import pandas as pd
import numpy as np
import random as rd

In [37]:
df = pd.read_csv("house-details.csv")

/tmp/ipykernel_67909/3025330690.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("house-details.csv")


In [38]:
df = df[["ST_NUM", "ROOF_STRUCTURE"]].dropna()

In [39]:
# df = df.sample(n = 1200) # CHANGE THIS FOR MORE AGENTS

In [40]:
df.loc[df["ROOF_STRUCTURE"] == "F - Flat", 'capacity'] = 0
df.loc[df["ROOF_STRUCTURE"] == "H - Hip", 'capacity'] = 0
df.loc[df["ROOF_STRUCTURE"] == "M - Mansard", 'capacity'] = 0
df.loc[df["ROOF_STRUCTURE"] == "G - Gable", 'capacity'] = 6450
df.loc[df["ROOF_STRUCTURE"] == "L - Gambrel", 'capacity'] = 7230
df.loc[df["ROOF_STRUCTURE"] == "S - Shed", 'capacity'] = 8750
df.loc[df["ROOF_STRUCTURE"] == "O - Other", 'capacity'] = 9160

df

,ST_NUM,ROOF_STRUCTURE,capacity
0,104.0,F - Flat,0.0
1,197.0,F - Flat,0.0
2,199.0,F - Flat,0.0
3,201.0,M - Mansard,0.0
4,203.0,M - Mansard,0.0
...,...,...,...
182233,18.0,L - Gambrel,7230.0
182234,14.0,H - Hip,0.0
182235,14.0,H - Hip,0.0
182236,12.0,H - Hip,0.0


In [41]:
df["street_num"] = df["ST_NUM"]
df = df.drop(["ST_NUM", "ROOF_STRUCTURE"], axis = 1)

In [42]:
df

,capacity,street_num
0,0.0,104.0
1,0.0,197.0
2,0.0,199.0
3,0.0,201.0
4,0.0,203.0
...,...,...
182233,7230.0,18.0
182234,0.0,14.0
182235,0.0,14.0
182236,0.0,12.0


In [43]:
df[df["capacity"] == 0] # NUMBER OF BAPS

,capacity,street_num
0,0.0,104.0
1,0.0,197.0
2,0.0,199.0
3,0.0,201.0
4,0.0,203.0
...,...,...
182231,0.0,22.0
182232,0.0,20.0
182234,0.0,14.0
182235,0.0,14.0


In [45]:
df[df["capacity"] != 0] # NUMBER OF BPPS

,capacity,street_num
42,6450.0,246.0
43,6450.0,244.0
55,6450.0,222.0
82,6450.0,421.0
89,6450.0,443.0
...,...,...
182204,6450.0,8.0
182211,6450.0,1.0
182224,6450.0,10.0
182233,7230.0,18.0


In [46]:
bpp = df[df["capacity"] != 0].reset_index()
bpp = bpp.drop("index", axis = 1)

In [47]:
bpp["street_num"].astype('int').to_csv("bpp-pos.csv") # CHANGE THE :100 BASED ON TOTAL - NUMBER OF BAP
bpp["capacity"].astype('int').to_csv("bpp-cap.csv") # IN THIS LINE TOO

In [48]:
bap = df[df["capacity"] == 0].reset_index()
bap = bap.drop(["index", "capacity"], axis = 1)

In [49]:
bap["street_num"].astype('int').to_csv("bap-pos.csv") # CHANGE THIS TO NUMBER OF BAP

In [50]:
# REMEMBER TO CHANGE THE NUMBER OF BAPS AND BPPS IN CONFIG.YAML TOO